## CSE455/555 - Intro to Pattern Recognition Problem Set 2: Linear Discriminant Functions and Support Vector Machines

## 1. Task-1

In [1]:
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold, cross_val_score

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28*28)
x_train = x_train[:6000] # reduced size as per piazza post in order to decrease runtime
y_train = y_train[:6000]
x_test = x_test.reshape(x_test.shape[0], 28*28)

In [3]:
sv_classifier = SVC(C=1.0, kernel='linear') # Using dot-product kernel
sv_classifier.fit(x_train, y_train)

SVC(kernel='linear')

In [4]:
predictions = sv_classifier.predict(x_test)

In [5]:
accuracy = np.sum(np.equal(y_test, predictions)) / len(y_test)
print("Accuracy = " + str(accuracy*100)+"%")

Accuracy = 91.0%


The model performed with an accuracy of 91% on 10% training data. Now, using cross validation, we hope to see an increase in the accuracy of our model.

In [6]:
kfold = KFold(n_splits=5)
model = SVC(C=1.0, kernel='linear')
results = cross_val_score(model, x_test, y_test, cv = kfold)
print(results.mean()*100)

91.41


## 2. Task-2

**Problem:** Identify the Lagrange dual problem of the given primal problem<br>
**Given Features:** $(x1,y1),...,(xN,yN)$, where y1,...,yN ∈ {−1,1}<br>
**To minimize:** $w^T.w+C\sum_{i=1}^N\xi_i$, where $xi_i$ is the error made by separating vector w on the given feature(xi, yi)<br>
**Subject to:** $y_i.(w^T.x_i)\geq1 - \xi_i$ and $\xi_i \geq 0$, for i = 1,...,N<br>

### Solution
**Optimization Problem:** The given problem is a minimization problem and hence it comes under the optimization problems. An optimization problem is the one where we minimize or maximize a given function by choosing the input values from a given set.<br>
The **Lagrangian dual problem** is obtained by forming the Lagrangian of a minimization problem by using nonnegative *Lagrange multipliers* to add the constraints to the objective function, and then solving for the primal variable values that *minimize the original* objective function.

The Lagrange function:<br>
$\mathcal{L}\ = \dfrac{1}{2}{\vec{w}}^T\vec{w}+\sum_{i}{\alpha_i\left(1-y_i.w^Tx_i\right)}$<br>
where $\alpha_i$ are the nonnegative Lagrange multipliers, i.e. $\alpha_i\geq0$, and<br>
inequslity constraints $(1-y_i.w^Tx_i\leq0)$, for 1 <= i <= $\mathcal{l}$<br>
Now, we set the partial derivative of Lagrange function over primal variable to 0: <br>
$\partial_w\mathcal{L} \implies \partial_w(\dfrac{1}{2}{\vec{w}}^T\vec{w}+\sum_{i}{\alpha_i\left(1-y_i.w^Tx_i\right)}) \implies w - \sum_{i}\alpha_iy_i\vec{x_i} = 0$<br>
We can write this as:<br>
$w = \sum_{i}\alpha_iy_i\vec{x_i}$
Here, we can see that the *Second order partial derivative* $\partial_w^2\mathcal{L} = 1 \geq0$<br>
This means that $w = \sum_{i}\alpha_iy_i\vec{x_i}$ minimizes $\mathcal{L}$ with given $\alpha_i\forall_i$<br><br>
$\mathcal{L}\ = \dfrac{1}{2}{\vec{w}}^T\vec{w}+\sum_{i}{\alpha_i\left(1-y_i.w^Tx_i\right)}$<br>
$\mathcal{L}\ =  + \sum_{i}{\alpha_i-w^T}\sum_{i}{\alpha_iy_i\vec{x_i}}$<br>
$\mathcal{L}\ = \sum_i\alpha_i - \dfrac{1}{2}{w}^T\sum_{i}{\alpha_iy_i\vec{x_i}}$<br>
$\mathcal{L}\ = \sum_i\alpha_i - \dfrac{1}{2}{w}^T\sum_{i, j}{\alpha_i\alpha_jy_iy_j\vec{x_i}^Tx_j}$<br>
where, <br>dual constraints $\alpha_i\geq0$, for all i and<br>
primal constraints $(1-y_i.w^Tx_i\leq0)$, for all i and<br>
complementary slackness $\alpha(1 - y_i.w^Tx_i) = 0$

For **Dual Formation**, margin = $\max_\alpha(\mathcal{L_{min}})$<br>
For **primal formulation**, margin = $\min_w\max_\alpha(\mathcal{L})$<br>
Hence, $margin_{dual} \leq margin_{primal}$

### Benefits of:
***- Maximizing the margin***<br>
***- Solving the dual problem instead of primal problem***<br>
The size of the margin defines the confidence of the classifier, hence the most wide margin is preferable. Hence, a classifier with a large margin makes no low certainty classification decisions. This will make sure that a slight error in measurement will not cause a mis-classification. Maximizing margins also leads to sparse set of support vectors<br>When we solve the primal problem, we obtain the optimal $w$, but we dont have informatin about the $\alpha_i$. Hence, in order to classify a query point x, we need to explicitly compute the scalar product of $w^Tx$, which may be expensive, especially in the higher dimension where d>>n.<br>
Previously, I have shown that the dual function gives a *lower bound* to the primal function. So, instead of solving the whole primal problem, we can solve for the *maximum lower bound* on the optimal value of the primal problem, by maximizing the Lagrangian Dual function, which is the dual problem.

After taking the partial derivative of the Lagrange function, we got:<br>
$w = \sum_{i}\alpha_iy_i\vec{x_i}$, which gives us the weight
Suppose the optimal solution is obtained by $\alpha^*, w^*$, we get the complimentary slackness as $\alpha^*(1 - y_i.\vec{w^*}^Tx_i) = 0$, for all i, again from the above equations.<br>
The discriminant function $f(\vec{x},\alpha^*)=\sum_{i\in{sv}}y_i\vec{\alpha_i}^*\vec{x_i}^Tx$, is a function of **support vectors:** $(\vec{x_i}, y_i)$, where $\alpha^* \neq 0$. For $i\in{sv}, y_if(\vec{x_i},\alpha^*)=1\implies \vec{w^*}^T\vec{w^*}=\sum_{i\in{sv}}\alpha_i^*$

## 3. Task-3

In its most simple form, SVM only supports binary classification and doesn't support multiclass classification. For multiclass, the problem is broken down into multiple binary classification problem.<br>
For this purpose, we use m-SVM model, where the support vector machine deals with all the categories simultaneously. We construct k SVM models for k classes, where the $t^{th}$ SVM has positive labels for the $t^{th}$ class and negatove for others.<br>

**Problem:** <br>Minimize: $\frac{1}{2}(w^m)^Tw^m+C\sum_{i=1}^l\xi_i^m$ ,on $(w^m,b^m,\xi^m)$ <br>
Subject to $(w^m)^T\phi(x_i)+b^m\geq1-\xi_i^m$, if $y_i=m$ (as it is the $m^{th}$ class), and
$(w^m)^T\phi(x_i)+b^m\leq-(1-\xi_i^m)$, if $y_i\neq m$<br>
for 1 <= i <= $\mathcal{l}$, $\xi_i^m \geq 0$ for a given set of features $(x_1, y_1),...,(x_\mathcal{l}, y_\mathcal{l})$ and $y_i$ is a set of classes from 1 to k

#### References:
 - Lecture Slides for Support Vector Machine (UBLearns)
 - https://www1.cmc.edu/pages/faculty/aaksoy/latex/latexthree.html# for LaTeX commands
 - https://en.wikipedia.org/wiki/Duality_(optimization)
 - https://en.wikipedia.org/wiki/Mathematical_optimization
 - https://www.baeldung.com/cs/svm-multiclass-classification
 - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
 - Pattern Classification - Richard Duda, Peter Hart and David G. Stork
 - Piazza - CSE455/555